In [1]:
# ===== SALES ANALYSIS PROJECT =====

# Part I: Extract, Transform, Load 

In [2]:
# Import libraries

import pandas as pd
import numpy as np

In [3]:
# Extract data from feather file and into a DataFrame

raw_sales_data = pd.read_feather(r"C:\Users\paulm\OneDrive\Desktop\Data Projects\4..+Sales+Analysis\Sales_data.ftr")

In [4]:
# Check first 10 rows of DataFrame to get familiar with the data
# NOTE: Items purchased together share the same Order ID, multiple rows can have the same Order ID
# NOTE: The 'Order ID' column will have duplicated values that must not be removed

raw_sales_data.head(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,None,None,None,None,None,None
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
6,176562,USB-C Charging Cable,1,11.95,04/29/19 13:03,"381 Wilson St, San Francisco, CA 94016"
7,176563,Bose SoundSport Headphones,1,99.99,04/02/19 07:46,"668 Center St, Seattle, WA 98101"
8,176564,USB-C Charging Cable,1,11.95,04/12/19 10:58,"790 Ridge St, Atlanta, GA 30301"
9,176565,Macbook Pro Laptop,1,1700,04/24/19 10:38,"915 Willow St, San Francisco, CA 94016"


In [5]:
# Use .shape to understand the size of the dataset

raw_sales_data.shape

(186850, 6)

In [6]:
# Check for null values

raw_sales_data.isna().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [7]:
# Drop null values

nonull_sales_data = raw_sales_data.dropna(how='all')

In [8]:
# Verify null values have been removed

nonull_sales_data.isna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [9]:
# Check for duplicate rows

nonull_sales_data.duplicated().sum()

618

In [10]:
# Drop duplicate rows
# NOTE: Using .drop_duplicates() only creates a slice of the DataFrame
# Copy the slice and assign it to new DataFrame 

nodup_sales_data = nonull_sales_data.drop_duplicates().copy()

In [11]:
# Verify all duplicate rows have been removed

nodup_sales_data.duplicated().sum()

0

In [12]:
# To load this DataFrame into MySQL, column values must be the correct data types
# Check data types

nodup_sales_data.dtypes

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
dtype: object

In [13]:
# For loop that will iterate through and convert numeric columns first
# The loop will then convert date column to datetime in the MM/DD/YYYY HH:MM format
# NOTE: Using (errors='coerce') which may result in some NaN values that need to be addressed

for col in nodup_sales_data.columns:
    try:
        if col in ['Order ID', 'Quantity Ordered', 'Price Each']:
            # Convert numeric columns
            nodup_sales_data[col] = pd.to_numeric(nodup_sales_data[col], errors='coerce')
        elif col == 'Order Date':
            # Convert date column with a specified format
            nodup_sales_data[col] = pd.to_datetime(nodup_sales_data[col], format='%m/%d/%y %H:%M', errors='coerce')
    except Exception as e:
        print(f"Could not convert column {col}: {e}")

In [14]:
# Verify data type conversions
# NOTE: 'Product' and 'Purchase Address' still need to be converted to string 

nodup_sales_data.dtypes

Order ID                   float64
Product                     object
Quantity Ordered           float64
Price Each                 float64
Order Date          datetime64[ns]
Purchase Address            object
dtype: object

In [15]:
# Check for rows with NaN values

nodup_sales_data[nodup_sales_data.isna().any(axis=1)]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
519,NaN,Product,NaN,NaN,NaT,Purchase Address


In [16]:
# Remove rows where 'Order ID' is NaN, as these rows contain incomplete data

nodup_sales_data = nodup_sales_data[nodup_sales_data['Order ID'].notna()]

In [17]:
# Check again for NaN values

nodup_sales_data.isna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [18]:
# Convert 'Product' and 'Purchase Address' to string 

nodup_sales_data['Product'] = nodup_sales_data['Product'].astype('string')
nodup_sales_data['Purchase Address'] = nodup_sales_data['Purchase Address'].astype('string')

In [19]:
# Check 'Product' data type conversion

nodup_sales_data['Product'][:5].dtypes

string[python]

In [20]:
# Check 'Purchase Address' data type conversion

nodup_sales_data['Purchase Address'][:5].dtypes

string[python]

In [21]:
# Check for NaN values

nodup_sales_data.isna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [22]:
# Remove any leading or trailing spaces from strings

nodup_sales_data['Product'] = nodup_sales_data['Product'].str.strip()
nodup_sales_data['Purchase Address'] = nodup_sales_data['Purchase Address'].str.strip()

In [23]:
# Parse 'Purchase Address' into 'Street', 'City', 'State_Zip'
nodup_sales_data[['Street', 'City', 'State_Zip']] = nodup_sales_data['Purchase Address'].str.rsplit(',', n=2, expand=True)

# Further split 'State_Zip' into 'State' and 'Zip'
nodup_sales_data[['State', 'Zip']] = nodup_sales_data['State_Zip'].str.extract(r'(\w+)\s+(\d+)')

# Drop the intermediate 'State_Zip' column
nodup_sales_data.drop('State_Zip', axis=1, inplace=True)

In [24]:
# Verify parsing output is correct

nodup_sales_data[['Street', 'City', 'State', 'Zip']].head()

,Street,City,State,Zip
0,917 1st St,Dallas,TX,75001
2,682 Chestnut St,Boston,MA,02215
3,669 Spruce St,Los Angeles,CA,90001
4,669 Spruce St,Los Angeles,CA,90001
5,333 8th St,Los Angeles,CA,90001


In [25]:
# Drop the 'Purchase Address' column, because its now redundant

nodup_sales_data.drop('Purchase Address', axis=1, inplace=True)

In [26]:
# Verify columns are correct

nodup_sales_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Street,City,State,Zip
0,176558.0,USB-C Charging Cable,2.0,11.95,2019-04-19 08:46:00,917 1st St,Dallas,TX,75001
2,176559.0,Bose SoundSport Headphones,1.0,99.99,2019-04-07 22:30:00,682 Chestnut St,Boston,MA,02215
3,176560.0,Google Phone,1.0,600.00,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001
4,176560.0,Wired Headphones,1.0,11.99,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001
5,176561.0,Wired Headphones,1.0,11.99,2019-04-30 09:27:00,333 8th St,Los Angeles,CA,90001


In [27]:
# Check for any NaN values after parsing

nodup_sales_data.isna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Street              0
City                0
State               0
Zip                 0
dtype: int64

In [28]:
# Check 'Quantity Ordered' and 'Price Each' for outliers
# Check min values are non-negative

nodup_sales_data[['Quantity Ordered', 'Price Each']].describe()

,Quantity Ordered,Price Each
count,185686.000000,185686.000000
mean,1.124544,184.519255
std,0.443069,332.843838
min,1.000000,2.990000
25%,1.000000,11.950000
50%,1.000000,14.950000
75%,1.000000,150.000000
max,9.000000,1700.000000


In [29]:
# Verify date range is within expected time frame

nodup_sales_data['Order Date'].min(), nodup_sales_data['Order Date'].max()

(Timestamp('2019-01-01 03:07:00'), Timestamp('2020-01-01 05:13:00'))

In [30]:
# Standardize column names by converting all characters to lowercase and replacing spaces with underscores

nodup_sales_data.columns = nodup_sales_data.columns.str.lower().str.replace(' ', '_')

In [31]:
# Verify output of standardization

nodup_sales_data.columns

Index(['order_id', 'product', 'quantity_ordered', 'price_each', 'order_date',
       'street', 'city', 'state', 'zip'],
      dtype='object')

In [32]:
# Add a 'total_price' column to facilitate analyses

nodup_sales_data['total_price'] = nodup_sales_data['quantity_ordered'] * nodup_sales_data['price_each']

In [33]:
# Verify 'total_price' column and calculation 

nodup_sales_data.head()

,order_id,product,quantity_ordered,price_each,order_date,street,city,state,zip,total_price
0,176558.0,USB-C Charging Cable,2.0,11.95,2019-04-19 08:46:00,917 1st St,Dallas,TX,75001,23.90
2,176559.0,Bose SoundSport Headphones,1.0,99.99,2019-04-07 22:30:00,682 Chestnut St,Boston,MA,02215,99.99
3,176560.0,Google Phone,1.0,600.00,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001,600.00
4,176560.0,Wired Headphones,1.0,11.99,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001,11.99
5,176561.0,Wired Headphones,1.0,11.99,2019-04-30 09:27:00,333 8th St,Los Angeles,CA,90001,11.99


In [34]:
# Check 'total_price' column for calculation errors by comparing summary statistics of 'total_price' vs 'price_each'

nodup_sales_data[['price_each','total_price']].describe()

,price_each,total_price
count,185686.000000,185686.000000
mean,184.519255,185.611936
std,332.843838,333.032118
min,2.990000,2.990000
25%,11.950000,11.950000
50%,14.950000,14.950000
75%,150.000000,150.000000
max,1700.000000,3400.000000


In [35]:
# As noted before 'order_id' values repeat because they are assigned to every product on the same order
# NOTE: 'order_id' column will have duplicate values that should not be removed they will facilitate aggregation

# Add a unique identifier column as 'row_id'
nodup_sales_data['row_id'] = range(1, len(nodup_sales_data) + 1)

# Verify the uniqueness, should return true
nodup_sales_data['row_id'].is_unique  

True

In [36]:
# Move 'row_id' to the leftmost position
cols = ['row_id'] + [col for col in nodup_sales_data.columns if col != 'row_id']
nodup_sales_data = nodup_sales_data[cols]

# Verify output
nodup_sales_data.head()

,row_id,order_id,product,quantity_ordered,price_each,order_date,street,city,state,zip,total_price
0,1,176558.0,USB-C Charging Cable,2.0,11.95,2019-04-19 08:46:00,917 1st St,Dallas,TX,75001,23.90
2,2,176559.0,Bose SoundSport Headphones,1.0,99.99,2019-04-07 22:30:00,682 Chestnut St,Boston,MA,02215,99.99
3,3,176560.0,Google Phone,1.0,600.00,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001,600.00
4,4,176560.0,Wired Headphones,1.0,11.99,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001,11.99
5,5,176561.0,Wired Headphones,1.0,11.99,2019-04-30 09:27:00,333 8th St,Los Angeles,CA,90001,11.99


In [37]:
# Data is cleaned, assign nodup_sales_data to clean_sales_data since no more modifications are needed

clean_sales_data = nodup_sales_data

In [38]:
# Below is the MySQL code used to create the schema and table in MySQL Workbench
# This is where clean_sales_data will be loaded

#  -- Create schema
#  DROP SCHEMA IF EXISTS sales_analysis;
#  CREATE SCHEMA IF NOT EXISTS sales_analysis;

#  -- Create table in MySQL
#  USE sales_analysis;

#  DROP TABLE IF EXISTS sales_data;
#  CREATE TABLE IF NOT EXISTS sales_data (
#      row_id INT PRIMARY KEY,
#      order_id INT,
#      product VARCHAR(255),
#      quantity_ordered INT,
#      price_each FLOAT,
#      order_date DATETIME,
#      street VARCHAR(255),
#      city VARCHAR(100),
#      state VARCHAR(10),
#      zip VARCHAR(20),
#      total_price FLOAT
# );

In [39]:
# Import libraries

import os
from sqlalchemy import create_engine

# Fetch environment variables
username = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
database = os.getenv('MYSQL_DB')

# Create sqlalchemy engine
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

# Load the clean_sales_data into MySQL
clean_sales_data.to_sql(
    'sales_data',       # Table name in MySQL
    con=engine,         # Connection to MySQL
    if_exists='replace', # Append to existing table, or use 'replace' to overwrite
    index=False         # Exclude the DataFrame index
)

185686

In [40]:
# Verify loading with SQL queries from Jupyter 

# Test Query 1: Check row count
query_1 = "SELECT COUNT(*) AS row_count FROM sales_data;"
row_count = pd.read_sql(query_1, engine)
print("Row Count:")
print(row_count)

# Test Query 2: Retrieve column names and sample rows
query_2 = "SELECT * FROM sales_data LIMIT 5;"
sample_data = pd.read_sql(query_2, engine)
print("Sample Data:")
print(sample_data)

# Test Query 3: Check date range
query_min_max_dates = """
SELECT 
    MIN(order_date) AS min_date, 
    MAX(order_date) AS max_date 
FROM sales_data;
"""

# Execute the Test Query 3
min_max_dates_sql = pd.read_sql(query_min_max_dates, engine)
print("Min and Max Dates from MySQL:")
print(min_max_dates_sql)

# Get min and max dates from clean_sales_data
min_date_local = clean_sales_data['order_date'].min()
max_date_local = clean_sales_data['order_date'].max()

# Display results
print("Min and Max Dates from clean_sales_data:")
print(f"Min Date: {min_date_local}")
print(f"Max Date: {max_date_local}")

Row Count:
   row_count
0     185686
Sample Data:
   row_id  order_id                     product  quantity_ordered  price_each  \
0       1  176558.0        USB-C Charging Cable               2.0       11.95   
1       2  176559.0  Bose SoundSport Headphones               1.0       99.99   
2       3  176560.0                Google Phone               1.0      600.00   
3       4  176560.0            Wired Headphones               1.0       11.99   
4       5  176561.0            Wired Headphones               1.0       11.99   

           order_date           street          city state    zip  total_price  
0 2019-04-19 08:46:00       917 1st St        Dallas    TX  75001        23.90  
1 2019-04-07 22:30:00  682 Chestnut St        Boston    MA  02215        99.99  
2 2019-04-12 14:38:00    669 Spruce St   Los Angeles    CA  90001       600.00  
3 2019-04-12 14:38:00    669 Spruce St   Los Angeles    CA  90001        11.99  
4 2019-04-30 09:27:00       333 8th St   Los Angeles    CA

In [41]:
# Test Query 4: Check NULL values in each column
query_null_counts = """
SELECT 
    SUM(CASE WHEN row_id IS NULL THEN 1 ELSE 0 END) AS row_id_nulls,
    SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS order_id_nulls,
    SUM(CASE WHEN product IS NULL THEN 1 ELSE 0 END) AS product_nulls,
    SUM(CASE WHEN quantity_ordered IS NULL THEN 1 ELSE 0 END) AS quantity_ordered_nulls,
    SUM(CASE WHEN price_each IS NULL THEN 1 ELSE 0 END) AS price_each_nulls,
    SUM(CASE WHEN order_date IS NULL THEN 1 ELSE 0 END) AS order_date_nulls,
    SUM(CASE WHEN street IS NULL THEN 1 ELSE 0 END) AS street_nulls,
    SUM(CASE WHEN city IS NULL THEN 1 ELSE 0 END) AS city_nulls,
    SUM(CASE WHEN state IS NULL THEN 1 ELSE 0 END) AS state_nulls,
    SUM(CASE WHEN zip IS NULL THEN 1 ELSE 0 END) AS zip_nulls,
    SUM(CASE WHEN total_price IS NULL THEN 1 ELSE 0 END) AS total_price_nulls
FROM sales_data;
"""

# Execute query and store results in DataFrame
null_counts = pd.read_sql(query_null_counts, engine)

# Display results
print("NULL Value Counts in MySQL Database:")
print(null_counts)

NULL Value Counts in MySQL Database:
   row_id_nulls  order_id_nulls  product_nulls  quantity_ordered_nulls  \
0           0.0             0.0            0.0                     0.0   

   price_each_nulls  order_date_nulls  street_nulls  city_nulls  state_nulls  \
0               0.0               0.0           0.0         0.0          0.0   

   zip_nulls  total_price_nulls  
0        0.0                0.0  


In [42]:
# Test Query 5: Get total sales and total quantity ordered from MySQL
query_totals_sql = """
SELECT 
    SUM(total_price) AS total_sales_sql, 
    SUM(quantity_ordered) AS total_quantity_sql
FROM sales_data;
"""

# Execute the query and load results into a DataFrame
totals_sql = pd.read_sql(query_totals_sql, engine)

# Display results
print("Totals from MySQL:")
print(totals_sql)

Totals from MySQL:
   total_sales_sql  total_quantity_sql
0     3.446554e+07            208812.0


In [43]:
# Get total sales and total quantity ordered from clean_sales_data
total_sales_local = clean_sales_data['total_price'].sum()
total_quantity_local = clean_sales_data['quantity_ordered'].sum()

# Display results
print("\nTotals from clean_sales_data:")
print(f"Total Sales: {total_sales_local}")
print(f"Total Quantity Ordered: {total_quantity_local}")


Totals from clean_sales_data:
Total Sales: 34465537.94
Total Quantity Ordered: 208812.0


In [44]:
# Loaded data looks good!
# Dispose of the engine

engine.dispose()  
print("Engine disposed, Extract, Transfrom, Load IS COMPLETE!!!.")

Engine disposed, Extract, Transfrom, Load IS COMPLETE!!!.
